Il k-nearest neighbors è un algoritmo di classificazione basato sull'assunto che le osservazioni simili appartengano alla stessa classe. Consiste nel calcolare la distanza tra un'osservazione di test e tutte le osservazioni del dataset di training e nell'assegnare l'etichetta della classe più presente tra i k osservazioni più vicine.

Il processo è questo:

1) Scegliere il numero di vicini k da considerare durante il calcolo delle distanze

2) Calcolare la distanza tra l'osservazione di test e tutte le osservazioni del dataset di training

3) Selezionare le k osservazioni più vicine

4) Assegnare l'etichetta della classe più presente tra le k osservazioni più vicine all'osservazione di test

Questo algoritmo può essere lento nel caso di dataset di grandi dimensioni e l'accuratezza del modello può essere influenzata in modo significativo dalla scelta del numero di vicini k.


Pro: 

Semplicità: è un algoritmo semplice e facile da capire e implementare

Buona prestazione con dataset di piccole dimensioni

Contro:

Lentezza con dataset di grandi dimensioni perchè deve calcolare la distanza tra l'osservazione di test e tutte le osservazioni ogni volta. Infatti è considerato un algoritmo lazy learner, ossia un algoritmo che richiede meno tempo per l'apprendimento e più tempo per la classificazione

Sensibilità alla scelta del numero k di vicini

Sensibilità alla presenza di outliers

In [2]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")
df <- df_finale

In [3]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

# Feature selection

Feature selection fatta attraverso l'algoritmo boruta che si basa sull'algoritmo di random forest e utilizza una tecnica di confronto dei risultati ottenuti con un insieme di dati di controllo per determinare quali variabili sono significative. è un algoritmo che tiene conto delle interazioni tra variabili, è versatile in quanto lavora sia su variabili continue che categoriche ed è resistente agli outliers

In [4]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


Warning message:
"package 'Boruta' was built under R version 3.6.3"
 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +1.9 mins: 

 confirmed 17 attributes: Admsite, Albumin, Ascites, Centre, CRIO and 12 more;

 rejected 39 attributes: Age, Alcoholic, Anticoagulation, Cavaveinanasth, Cholestatic and 34 more;

 still have 17 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +2.2 mins: 

 confirmed 1 attribute: HCC;

 rejected 6 attri

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [5]:
par(mar = c(15, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.1, las=2, xlab="", main="Variable Importance")

In [6]:
df_boruta$deg_cat <- df$deg_cat

In [7]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Nocomorbodities"  "Admsite"          "Encephalopathy"  
[25] "HCC"

## Data partition

Viene effettuata divisione tra training e test set, che è del 70-30 per tutti i modelli utilizzati

In [8]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)



Viene effettuato un bilanciamento della variabile target perchè in origine la classe ottimo conteneva circa 1/4 dei record rispetto alle altre due classi e i modelli facevano fatica a classificare quel determinato valore. La funzione utilizzata è SmoteClassif() che:

1) Seleziona a caso una osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

2) Seleziona a caso un'altra osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

3) Genera una nuova osservazione sintetica che si trova a metà strada tra le due osservazioni selezionate in precedenza

4) Ripete i passaggi precedenti fino a raggiungere il bilanciamento desiderato


Questa funzione utilizza il knn per calcolare le distanze tra le osservazioni e generare le osservazioni, infatti il valore di k è stato selezionato pari alla radice quadrata del numero totale di record presenti nel training set come si fa per il knn normalmente per il knn



In [9]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    488     488 

Viene effettuato lo stesso processo considerando il dataset dopo la boruta feature selection invece del dataset con tutte le feature in modo da poter fare una comparazione sui risultati successivamente

In [10]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_boruta$deg_cat, SplitRatio = 0.7)
trainB  <- subset(df_boruta, sample == TRUE)
testB   <- subset(df_boruta, sample == FALSE)



TestClassesB <- testB$deg_cat
TestDataB <- subset(testB,select= -deg_cat)


In [11]:
trainB <- SmoteClassif(deg_cat ~ ., trainB, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(trainB$deg_cat)

TrainClassesB <- trainB$deg_cat
TrainDataB <- subset(trainB,select= -deg_cat)


   fast NotFast 
    488     488 

## Models

Attraverso la funzione trainControl viene effettuata una repeated cross validation e una random search.

La prima permette di ottenere una stima più accurata dell'accuratezza del modello:

1) Dividere il dataset in un numero predeterminato di folds (in questo caso è stato scelto n=10 quindi 10 folds)

2) Eseguire la cross-validation ripetutamente utilizzando ogni fold come test set e gli altri folds come training set

3) Calcolare la media e la deviazione standard delle misure di accuratezza ottenute per ogni fold

4) Valutare l'accuratezza e la robustezza del modello utilizzando la media e la deviazione standard delle misure di accuratezza

L'utilizzo della repeated cross validation è particolarmente importante quando si lavora con dataset di dimensioni ridotte o con un numero limitato di osservazioni perchè in questi casi i risultati possono essere influenzata dall'aleatorietà nella divisione dei dati in training set e test set. La repeated cross-validation permette quindi di ottenere una stima più accurata dell'accuratezza del modello ripetendo il processo  più volte e considerando dati diversi



La random search, invece è una tecnica di ottimizzazione dei modelli che viene utilizzata per selezionare i migliori parametri di un modello. Consiste nel generare a caso diverse combinazioni di parametri e nel valutare l'accuratezza del modello per ogni combinazione. La combinazione di parametri che ottiene il risultato migliore viene quindi selezionata come quella ottimale. Questa soluzione è stata preferita alla grid search che poteva dare risultati migliori per una questione di costo computazionale

In [12]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

Viene allenato il modello knn sul dataset senza feature selection

In [13]:
model_knn <- train(x=TrainData, y=TrainClasses, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
- Fold01.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
+ Fold01.Rep1: kmax=315, distance=2.757, kernel=biweight 
- Fold01.Rep1: kmax=315, distance=2.757, kernel=biweight 
+ Fold01.Rep1: kmax=293, distance=2.342, kernel=epanechnikov 
- Fold01.Rep1: kmax=293, distance=2.342, kernel=epanechnikov 
+ Fold02.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
- Fold02.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
+ Fold02.Rep1: kmax=315, distance=2.757, kernel=biweight 
- Fold02.Rep1: kmax=315, distance=2.757, kernel=biweight 
+ Fold02.Rep1: kmax=293, distance=2.342, kernel=epanechnikov 
- Fold02.Rep1: kmax=293, distance=2.342, kernel=epanechnikov 
+ Fold03.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
- Fold03.Rep1: kmax=267, distance=1.288, kernel=epanechnikov 
+ Fold03.Rep1: kmax=315, distance=2.757, kernel=biweight 
- Fold03.Rep1: kmax=315, distance=2.757, kernel=biweight 
+ Fold03.Rep1: kmax=293, distanc

Viene allenato il modello knn sul dataset con feature selection

In [14]:
model_knnB <- train(x=TrainDataB, y=TrainClassesB, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
- Fold01.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
+ Fold01.Rep1: kmax=211, distance=2.13645, kernel=triangular 
- Fold01.Rep1: kmax=211, distance=2.13645, kernel=triangular 
+ Fold01.Rep1: kmax=109, distance=0.02771, kernel=triweight 
- Fold01.Rep1: kmax=109, distance=0.02771, kernel=triweight 
+ Fold02.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
- Fold02.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
+ Fold02.Rep1: kmax=211, distance=2.13645, kernel=triangular 
- Fold02.Rep1: kmax=211, distance=2.13645, kernel=triangular 
+ Fold02.Rep1: kmax=109, distance=0.02771, kernel=triweight 
- Fold02.Rep1: kmax=109, distance=0.02771, kernel=triweight 
+ Fold03.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
- Fold03.Rep1: kmax= 55, distance=0.08825, kernel=triangular 
+ Fold03.Rep1: kmax=211, distance=2.13645, kernel=triangular 
- Fold03.Rep1: kmax=211, distance=2.13645, kernel=triangular 
+ Fold03.Rep

I parametri testati sono: 

kmax = numero di nearest neighbors considerati

distance = parametro per la distanza minkowski

kernel = per specificare il metodo di con cui vengono calcolati i pesi da utilizzare durante il calcolo delle distanze tra le osservazioni

## Results

Vengono analizzati i risultati ottenuti dai due modelli durante la cross validation. Le metriche utilizzate sono: accuracy e kappa. La prima si riferisce alla percentuale di osservazioni correttamente classificate dal modello rispetto al totale di osservazioni. La seconda viene utilizzata per misurare l'accordo tra più modelli di classificazione. Può assumere valori compresi tra -1 e 1. Un valore di kappa vicino a 1 indica un alto livello di accordo tra le classificazioni, mentre un valore di kappa vicino a 0 indica un accordo simile a quello ottenuto per caso. Un valore di kappa negativo indica un accordo peggiore di quello ottenuto per caso.

In [15]:
print(model_knn)
plot(model_knn)

k-Nearest Neighbors 

976 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 878, 878, 878, 878, 878, 879, ... 
Resampling results across tuning parameters:

  kmax  distance  kernel        Accuracy   Kappa    
  267   1.288433  epanechnikov  0.7047674  0.4093896
  293   2.342255  epanechnikov  0.6976142  0.3951351
  315   2.756687  biweight      0.6924589  0.3848638

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 267, distance = 1.288433
 and kernel = epanechnikov.


In [16]:
print(model_knnB)
plot(model_knnB)

k-Nearest Neighbors 

976 samples
 25 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 878, 879, 879, 878, 879, 879, ... 
Resampling results across tuning parameters:

  kmax  distance    kernel      Accuracy   Kappa    
   55   0.08824917  triangular  0.7367768  0.4734431
  109   0.02770526  triweight   0.7120450  0.4240424
  211   2.13645253  triangular  0.7827477  0.5653893

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 211, distance = 2.136453
 and kernel = triangular.


Vengono messi a confronto i risultati ottenuti sulla cross validation dei due modelli in modo da selezionare il più performante da utilizzare nel test set

In [17]:
# collect resamples
results <- resamples(list(knn=model_knn,knn_boruta=model_knnB))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: knn, knn_boruta 
Number of resamples: 10 

Accuracy 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.5937500 0.6760204 0.6974542 0.7047674 0.7500000 0.7835052    0
knn_boruta 0.7142857 0.7654639 0.7755102 0.7827477 0.8020724 0.8469388    0

Kappa 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.1875000 0.3520408 0.3937922 0.4093896 0.5000000 0.5677912    0
knn_boruta 0.4285714 0.5313208 0.5510204 0.5653893 0.6044255 0.6938776    0


## Prediction

Viene selezionato il modello più performante per usarlo nel test set ed avere risultati definitivi.

In [18]:
pred_knn <- predict(model_knnB,test)  
pred_knn.prob <- predict(model_knnB,test, type="prob")


result <- confusionMatrix(TestClasses,  pred_knn) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     110      36
   NotFast   98     175
                                          
               Accuracy : 0.6802          
                 95% CI : (0.6332, 0.7246)
    No Information Rate : 0.5036          
    P-Value [Acc > NIR] : 1.885e-13       
                                          
                  Kappa : 0.359           
                                          
 Mcnemar's Test P-Value : 1.367e-07       
                                          
            Sensitivity : 0.5288          
            Specificity : 0.8294          
         Pos Pred Value : 0.7534          
         Neg Pred Value : 0.6410          
             Prevalence : 0.4964          
         Detection Rate : 0.2625          
   Detection Prevalence : 0.3484          
      Balanced Accuracy : 0.6791          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5288462            0.8293839            0.7534247 
      Neg Pred Value            Precision               Recall 
           0.6410256            0.7534247            0.5288462 
                  F1           Prevalence       Detection Rate 
           0.6214689            0.4964200            0.2625298 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.6791150

In [19]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_knn))
roc_knn



Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6972

In [20]:
roc_obj <- roc(TestClasses, as.numeric(pred_knn))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [21]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) in 146 controls (TestClasses fast) < 273 cases (TestClasses NotFast).
Area under the curve: 0.6972

Probabilità per ogni classe

In [22]:
pred_knn.prob

fast,NotFast
0.89874564,0.10125436
0.01619293,0.98380707
0.23695173,0.76304827
0.65070939,0.34929061
0.15975528,0.84024472
0.25108249,0.74891751
0.41869151,0.58130849
0.56421350,0.43578650
0.06855510,0.93144490
0.09515538,0.90484462
